In [15]:
# import os, datetime
# import github
# from progress.bar import ShadyBar
# from progress.spinner import Spinner
# import secrets
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from flask import Flask
from model import (Repo, User, Follower,
                   Stargazer, Watcher, Contributor,
                   Language, RepoLanguage,
                   db, connect_to_db, db_uri)
import utils

In [7]:
app = Flask(__name__)
connect_to_db(app)

In [8]:
# users = User.query.all()
# users_list = [[user.user_id, user.login, user.name]  for user in users]

# repos = Repo.query.all()
# repos_list = [[repo.repo_id, repo.name] for repo in repos]
# repos_df = pd.DataFrame(repos_list, columns = ["repo_id", "repo_name"])

ratings = Stargazer.query.all()
ratings_list = [[rating.user_id, rating.repo_id, 1] for rating in ratings]
ratings_df = pd.DataFrame(ratings_list, columns = ["user_id", "repo_id", "Rating"], dtype = int)
R_df = ratings_df.pivot(index="user_id", columns = "repo_id", values = "Rating").fillna(0)
R = R_df.as_matrix()

In [34]:
R_df.head()

repo_id,46939,65252,71972,516310,734934,759150,775892,858127,861039,943149,...,105616875,107457891,108084364,108252892,109536787,110424613,117445700,118370957,119348173,120153657
user_id,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
R_df.index

Int64Index([      17,      160,      224,      547,     2644,     2882,
                3045,     3624,     3856,     3879,
            ...
            33871080, 34081483, 34136216, 34445815, 34733638, 34776501,
            34831863, 35255957, 35267512, 35526196],
           dtype='int64', name='user_id', length=1862)

In [9]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [115]:
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
predictions_df = pd.DataFrame(predicted_ratings, columns = R_df.columns, index = R_df.index)

In [116]:
# print(user_ratings_mean)
# print(R_demeaned)
# print(sigma)
predictions_df.head()

repo_id,46939,65252,71972,516310,734934,759150,775892,858127,861039,943149,...,105616875,107457891,108084364,108252892,109536787,110424613,117445700,118370957,119348173,120153657
user_id,,,,,,,,,,,,,,,,,,,,,
17,-5.984796e-17,-1.040834e-17,-1.778092e-17,-8.673617e-18,1.257675e-17,-1.431147e-17,-7.806256e-18,-8.500145e-17,-2.602085e-17,1.734723e-17,...,-1.864828e-17,3.382711e-17,-6.505213e-18,-1.951564e-17,-2.732189e-17,1.084202e-17,-1.734723e-17,-1.908196e-17,-7.068998e-17,3.165870e-17
160,-1.301043e-18,-2.428613e-17,-3.382711e-17,-4.900594e-17,1.474515e-17,9.107298e-18,1.344411e-17,-5.204170e-17,-1.387779e-17,-7.372575e-18,...,7.806256e-18,9.540979e-18,-1.474515e-17,-6.722053e-17,9.497611e-17,1.604619e-17,-6.505213e-17,-2.125036e-17,-1.170938e-17,-3.816392e-17
224,-5.984796e-17,-1.040834e-17,-1.778092e-17,-8.673617e-18,1.257675e-17,-1.431147e-17,-7.806256e-18,-8.500145e-17,-2.602085e-17,1.734723e-17,...,-1.864828e-17,3.382711e-17,-6.505213e-18,-1.951564e-17,-2.732189e-17,1.084202e-17,-1.734723e-17,-1.908196e-17,-7.068998e-17,3.165870e-17
547,-4.857226e-17,-1.717376e-16,-2.602085e-18,-1.127570e-17,2.602085e-17,-2.688821e-17,-6.938894e-18,-1.032160e-16,-5.290907e-17,7.806256e-18,...,1.509209e-16,5.204170e-18,-1.474515e-17,-5.551115e-17,-2.341877e-17,1.214306e-17,1.144917e-16,-3.729655e-17,1.474515e-17,1.344411e-16
2644,-5.984796e-17,-1.040834e-17,-1.778092e-17,-8.673617e-18,1.257675e-17,-1.431147e-17,-7.806256e-18,-8.500145e-17,-2.602085e-17,1.734723e-17,...,-1.864828e-17,3.382711e-17,-6.505213e-18,-1.951564e-17,-2.732189e-17,1.084202e-17,-1.734723e-17,-1.908196e-17,-7.068998e-17,3.165870e-17


In [134]:
# predictions_df[predictions_df < 1].iloc[77].sort_values(ascending=False)
predictions_df.iloc[77].sort_values(ascending=False)

repo_id
35491164     1.000000e+00
4065801      5.134781e-16
8316778      1.214306e-16
109536787    9.497611e-17
11857350     9.454243e-17
55684118     8.500145e-17
88443003     7.936360e-17
10614163     7.806256e-17
32163997     7.199102e-17
103645588    6.808790e-17
2325298      6.418477e-17
70776647     6.114900e-17
59474324     6.071532e-17
53655574     5.984796e-17
89766385     5.507747e-17
7705639      5.117434e-17
35739817     5.030698e-17
3668353      5.030698e-17
10620937     4.553649e-17
28028078     4.380177e-17
12153393     4.336809e-17
11204323     4.250073e-17
23070145     4.206704e-17
26441713     4.163336e-17
30452474     4.119968e-17
39327533     4.076600e-17
22570419     3.989864e-17
21731829     3.816392e-17
22249781     3.816392e-17
57291806     3.816392e-17
                 ...     
60512478    -5.854692e-17
4160039     -5.941428e-17
6041301     -5.984796e-17
15655253    -6.028164e-17
57195766    -6.158268e-17
47803364    -6.331741e-17
117445700   -6.505213e-17
1082

In [117]:
predictions_df.describe()

repo_id,46939,65252,71972,516310,734934,759150,775892,858127,861039,943149,...,105616875,107457891,108084364,108252892,109536787,110424613,117445700,118370957,119348173,120153657
count,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,...,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03,1.862000e+03
mean,5.370569e-04,1.074114e-03,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,...,1.611171e-03,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04,5.370569e-04
std,2.317449e-02,3.276487e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,...,4.011782e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02,2.317449e-02
min,-9.436896e-16,-8.881784e-16,-5.104424e-16,-9.992007e-16,-7.658804e-16,-3.330669e-16,-3.191891e-16,-8.326673e-16,-4.371503e-16,-1.206066e-15,...,-1.665335e-15,-1.054712e-15,-1.165734e-15,-8.881784e-16,-9.506285e-16,-6.661338e-16,-1.221245e-15,-8.929489e-16,-4.024558e-16,-1.054712e-15
25%,-4.033232e-17,-1.040834e-17,-1.778092e-17,-8.673617e-18,-1.734723e-18,-2.038300e-17,-7.806256e-18,-5.204170e-17,-1.387779e-17,-8.239937e-18,...,-4.770490e-18,-1.079865e-16,-1.170938e-17,-2.081668e-17,-2.732189e-17,-3.859760e-17,-4.423545e-17,-2.125036e-17,-3.989864e-17,-3.816392e-17
50%,-1.301043e-18,-6.938894e-18,-3.903128e-18,1.778092e-17,1.257675e-17,-1.431147e-17,1.344411e-17,4.336809e-18,1.908196e-17,-3.903128e-18,...,7.806256e-18,9.540979e-18,-6.505213e-18,-1.908196e-17,5.204170e-18,-4.336809e-18,-1.734723e-17,-1.908196e-17,-2.298509e-17,3.165870e-17
75%,6.071532e-18,1.131907e-16,2.211772e-17,2.558717e-17,1.691355e-17,9.107298e-18,3.512815e-17,1.778092e-17,5.637851e-17,1.734723e-17,...,6.288373e-17,1.994932e-17,1.257675e-17,8.673617e-18,2.732189e-17,1.084202e-17,2.168404e-18,3.903128e-18,-1.170938e-17,3.426079e-17
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
